# A. Install Great Expectation Package

In [2]:
# Install the library

!pip install -q great-expectations

# B. Instantiate Data Context

In [3]:
# Create a data context

from great_expectations.data_context import FileDataContext

context = FileDataContext.create(project_root_dir='./')

# C. Connect to A `Datasource`

In [4]:
# Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'data_superstore' # namanya harus beda-beda
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = '_superstore' #beda-beda juga
path_to_data = 'dags\P2M3_vicky_eldora_data_clean.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

# D. Create an Expectation Suite

In [5]:
# Creat an expectation suite
expectation_suite_name = 'expectation-superstore-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,Unnamed: 0,unnamed_0,id,year_birth,education,marital_status,income,kidhome,teenhome,dt_customer,...,amount_fish_products,amount_sweet_products,amount_gold_prods,number_deals_purchases,number_web_purchases,number_catalog_purchases,number_store_purchases,number_web_visits_month,response,complain
0,0,0,1826,1970,Graduation,Divorced,84835.0,0,0,2014-06-16,...,111.0,79.75,127.75,1,4,4,6,1,1,0
1,1,1,1,1961,Graduation,Single,57091.0,0,0,2014-06-15,...,7.0,0.00,37.00,1,7,3,7,5,1,0
2,2,2,10476,1958,Graduation,Married,67267.0,0,1,2014-05-13,...,15.0,2.00,30.00,1,3,2,5,2,0,0
3,3,3,1386,1967,Graduation,Together,32474.0,1,1,2014-11-05,...,0.0,0.00,0.00,1,1,0,2,7,0,0
4,4,4,5371,1989,Graduation,Single,21474.0,1,0,2014-08-04,...,11.0,0.00,34.00,2,3,1,2,7,1,0


## E. - Expectations

In [7]:
# Expectation 1 : Column `id` must be unique
# The id column is used to uniquely identify each customer. There cannot be two customers with the same id.

validator.expect_column_values_to_be_unique('id')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 2064,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [8]:
# Expectation 2 : Column `year_birth` must be between 1893 and 1996
# The year_birth column must indicate the realistic birth year of the customer, i.e. between 1893 and 1996.

validator.expect_column_values_to_be_between(
    column='year_birth', min_value=1893, max_value=1996
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 2064,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [9]:
# Expectation 3 : Column `response` must contain one of the following 2 things :
# 1 = customer accepted the offer in the last campaign
# 0 = customer not accepted the offer in the last campaign
# The response column indicates whether the customer accepted or did not accept the campaign offer, with a value of 1 or 0.

validator.expect_column_values_to_be_in_set('response', [0, 1]) 

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 2064,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [12]:
# Expectation 4 : Column `income` must in form of integer or float
# The income column must be in the form of a number (integer or float) because it is not possible for income to be in the form of a string

validator.expect_column_values_to_be_in_type_list('income', ['int','float'])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "float64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [13]:
# Expectation 5: Column `dt_customer` must match datetime format %Y-%m-%d
# The join_date column must match the desired date format. i.e. year month and date

validator.expect_column_values_to_match_strftime_format(
    column='dt_customer', strftime_format='%Y-%m-%d'
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 2064,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [32]:
# Expectation 6: Column `income` values should be non-negative
validator.expect_column_min_to_be_between(
    column='income',
    min_value=0
)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 5305.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [29]:
# Expectation 7: Columns `id` and `response` values should be unique within each record
validator.expect_select_column_values_to_be_unique_within_record(
    column_list=['id', 'dt_customer']
)

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 2064,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Problem Statement :
Superstore mengalami penurunan jumlah customer, yang berdampak negatif pada penjualan dan pertumbuhan toko secara keseluruhan. Maka dari itu perusahaan ingin memahami penyebab dari penurunan ini, dengan cara menganalisis profil customer dan tanggapan mereka terhadap marketing campaign, sangat penting untuk mengoptimalkan strategi pemasaran di masa depan.

Objective :
Tujuan utamanya adalah untuk melakukan Analisis Data Eksplorasi (EDA) menggunakan Kibana pada data customer yang dikumpulkan selama tiga tahun terakhir dari marketing campaign. Menganalisis data ini bertujuan untuk lebih mengetahui segmentasi customer, efektivitas kampanye, dan perilaku pembelian. Wawasan ini akan membantu tim marketing dalam menyesuaikan kampanye di masa depan yang pada akhirnya dapat membalikkan tren penurunan kunjungan customer. Proyek ini memanfaatkan database NoSQL dan menggunakan Elasticsearch untuk menyederhanakan dan mengotomatiskan proses analisis data.

Dashboard panel
- Diagram lingkaran menunjukkan bahwa sebagian besar customer (97,69%) tidak menerima penawaran dari kampanye terakhir, sementara hanya sebagian kecil (2,31%) yang menerimanya. Hal ini mengindikasikan bahwa marketing campaign tersebut memiliki tingkat penerimaan yang rendah di antara para customer.
- Tingkat penerimaan yang rendah dapat disebabkan oleh berbagai faktor seperti relevansi penawaran dengan customer, efektivitasmarketing message, atau waktu kampanye.

- Diagram garis menunjukkan tren respon customer selama tiga tahun, dari tahun 2012 hingga 2014.
- Terdapat peningkatan yang signifikan dalam jumlah customer sekitar bulan Agustus 2012 dan Februari 2013. Hal ini mungkin disebabkan oleh marketing campaign yang sukses, permintaan musiman, atau kegiatan promosi lainnya selama periode tersebut.
- Sebaliknya, terjadi penurunan tajam jumlah nasabah pada Februari 2014 dan Juli 2014. Tren ini mungkin mengindikasikan efek musiman, kejenuhan pasar, atau upaya pemasaran yang kurang efektif selama periode tersebut.
Mengidentifikasi marketing campaign tertentu atau peristiwa eksternal yang bertepatan dengan lonjakan dan penurunan ini dapat membantu memahami tren ini dengan lebih baik dan meningkatkan strategi kampanye di masa depan.

- Histogram menunjukkan distribusi usia customer berdasarkan tahun kelahiran mereka.
-- customer tertua lahir pada tahun 1892, dan yang termuda lahir pada tahun 1996.
-- Mayoritas nasabah lahir antara awal tahun 1970-an hingga akhir tahun 1970-an, khususnya sekitar tahun 1970 hingga 1978. Hal ini menunjukkan bahwa sebagian besar basis customer berusia paruh baya.
Memahami distribusi usia dapat membantu menyesuaikan strategi pemasaran dengan kelompok usia yang dominan. Sebagai contoh, nasabah yang lahir pada tahun 1970-an mungkin merespons lebih baik terhadap penawaran dan pesan yang sesuai dengan tahap kehidupan dan minat mereka.

Pendidikan yang paling umum adalah Sarjana dengan persentase 50,47%. Hal ini menunjukkan bahwa sebagian besar individu dalam data memiliki gelar sarjana.

Tingkat pendidikan lainnya adalah:

Doktoral: 21,24%
Master 16.47%
Siklus ke-2: 9,21%
Dasar: 2,6


- Status pernikahan yang paling umum adalah Menikah dengan persentase 39,6%. Rentang tahun kelahiran yang paling umum untuk status ini adalah 1974, 1975, dan 1972.
- Status Menikah Bersama juga cukup tinggi dengan 25,93%. Rentang tahun kelahiran yang paling umum untuk status ini adalah 1976, 1978, dan 1969.
- Status perkawinan tunggal memiliki persentase 19,55%. Rentang tahun kelahiran yang paling umum untuk status ini adalah 1972, 1970, dan 1975.
- Status perkawinan Bercerai memiliki persentase 14,91%. Rentang tahun kelahiran yang paling umum untuk status ini adalah 1976, 1968, dan 1970.

- Status pernikahan Menikah memiliki jumlah pembelian tertinggi dibandingkan dengan status pernikahan lainnya. Hal ini wajar karena kebutuhan dan pengeluaran biasanya lebih banyak untuk individu yang sudah menikah.
- Kategori barang yang paling banyak dibeli oleh individu yang sudah menikah adalah Wine dan Daging.



Dari metrik tersebut dapat dilihat bahwa yang paling banyak adalah store purchases yang berarti pembelian yang dilakukan secara langsung di toko, kemudian web visit month yang berarti kunjungan ke website perusahaan dalam satu bulan terakhir, lalu web purchases yang berarti pembelian yang dilakukan melalui website perusahaan, kemudian catalog purchases yang berarti pembelian yang dilakukan dengan menggunakan katalog (membeli barang yang akan dikirim melalui pos), dan terakhir deals purchases yang berarti pembelian yang dilakukan dengan potongan harga.

Kesimpulan :
Rata-rata customer berusia paruh baya, lebih suka berbelanja di toko fisik, dan paling menyukai produk wine. Pendidikan yang paling umum di antara mereka adalah sarjana.

Wawasan :
1. Tingkat Pendidikan:

Lulus: Karena sebagian besar customer berpendidikan tinggi, marketing campaign dapat difokuskan pada produk premium yang relevan dengan gaya hidup mereka, seperti anggur berkualitas tinggi dan paket daging eksklusif.

2. Status Perkawinan:

Menikah: customer yang sudah menikah memiliki jumlah pembelian tertinggi. Untuk meningkatkan pembelian, tawarkan paket bundel atau diskon khusus untuk produk wine dan daging yang merupakan produk favorit mereka.

Bersama & Lajang: Tawarkan diskon untuk produk yang lebih kecil atau paket percobaan untuk menarik customer dengan status ini. Mungkin juga menarik untuk mengadakan acara komunitas atau lokakarya memasak untuk melibatkan mereka lebih jauh.

3. Usia:

customer Lanjut Usia: Sediakan program hadiah atau penghargaan untuk customer yang lebih tua (misalnya, diskon ulang tahun khusus atau poin loyalitas ekstra) untuk menghargai kesetiaan mereka.

customer yang lebih muda: Berikan diskon dan promosi khusus untuk menarik customer yang lebih muda, seperti diskon pelajar atau paket bundel produk yang sesuai dengan minat dan kebutuhan mereka.

4. marketing campaign:

Meningkatkan efektivitas marketing campaign dengan menyesuaikan pesan dan penawaran sesuai dengan segmen nasabah berdasarkan usia, status pernikahan, dan tingkat pendidikan.

Mengevaluasi dan menganalisis lebih lanjut marketing campaign yang gagal untuk mengidentifikasi faktor-faktor yang menyebabkan rendahnya tingkat penerimaan dan melakukan penyesuaian yang diperlukan.

5. Pembelian di Toko: ㆍ Karena pembelian di toko fisik adalah yang paling banyak, tingkatkan pengalaman berbelanja di dalam toko dengan layanan customer yang lebih baik, penataan produk yang menarik, dan promosi di dalam toko.attractive product arrangement, and in-store promotions.